In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [4]:
from implementations import *

## Do your thing crazy machine learning thing here :) ...

In [5]:
# Separate the data according to the value of column 24 (PRI_jet_num) 

def separate(y, tX, ids):
    
    split_x = []
    split_y = []
    split_ids = []
    
    jet_column_nbr = 22
    
    for i in range(4):
        
        split_x.append(tX[np.where(tX[:,jet_column_nbr] == i)])
        split_y.append(y[np.where(tX[:,jet_column_nbr] == i)])
        split_ids.append(ids[np.where(tX[:,jet_column_nbr] == i)])
    
    
    
    return split_x, split_y, split_ids

In [6]:
split_x, split_y, split_ids = separate(y, tX, ids)

In [7]:
#remove the columns from each set of data given a boolean array

def removeNone(data, selection):
   
    cleaned=[]
    
    for i in range(4):
        curr_data = data[i]
        
        cleaned.append(curr_data[:,selection[i]])
      
    return cleaned
    
    

In [8]:
#print statistics about the None values (-999) for each columns
#returns a boolean array that can be used to filter the columns that have 100% of undefined values (-999)
def dataStatistics(data):
    
    stats=[]
    
    for i in range(4):
        
        print("Statistics ")
        print("Type :")
        print(i)
        
        
        nones = (data[i] == -999)
    
        mean = np.sum(nones, axis=0)/nones.shape[0]
        print(mean) 
        stats.append(mean != 1)
    
    return stats
    

In [9]:
selection = dataStatistics(split_x)

Statistics 
Type :
0
[0.26145747 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.
 1.         1.         1.         1.         1.         0.        ]
Statistics 
Type :
1
[0.09751883 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         0.        ]
Statistics 
Type :
2
[0.05859584 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
Statistics 
Type :
3
[0.0

In [10]:
cleaned = removeNone(split_x, selection)



# Now we can either drop the lines with residual Nones or replace the Nones by the median value

In [11]:
#replace the value of column 0 (can be None sometimes) by the median value of this column

def putMedianInsteadOfNone(cleaned):
    
    completed_data = []
    
    for i in range(4):
        #current PRI_jet_num
        current = cleaned[i]
        
        median = np.median(current[np.where(current[:,0] != -999)], axis = 0)
        
        #replace -999 by median value
        current[np.where(current[:,0] == -999)] = median
        
        completed_data.append(current)
    
    
    return completed_data
    
    

In [12]:
cleaned_with_median = putMedianInsteadOfNone(cleaned)



In [12]:
#Instead of putting the median we can simply drop the data where columns 0 == -999
def dropLineIfNone(cleaned, split_y, split_ids):
    
    res_x=[]
    res_y=[]
    res_ids=[]
    
    for i in range(4):
        
        current = cleaned[i]
        
        drop_indexes = np.where(current[:,0] != -999)
        
        res_x.append(current[drop_indexes])
        res_y.append(current[drop_indexes])
        res_ids.append(current[drop_indexes])
        
    return res_x, res_y, res_ids

In [13]:
dropped_x, dropped_y, dropped_ids = dropLineIfNone(cleaned, split_y, split_ids)


## At this point, the first values in each of the split data has a PRI_jet_num = 0, then 1 and so on. The data is clean and we can work with it.

In [14]:
initial_w = np.zeros(cleaned_with_median[0].shape[1])
median_cleaned_x_0_lsq_GD = least_squares_GD(split_y[0], normalize(cleaned_with_median[0]), initial_w, 500, 0.1)

In [15]:
print(median_cleaned_x_0_lsq_GD)

(array([ 0.2155689 , -0.53554557, -0.40401874,  0.03959931,  0.08147027,
        0.03959934,  0.16148251,  0.03462872,  0.10957412,  0.5842968 ,
        0.05218412,  0.04930115, -0.04715568,  0.05726908,  0.0508691 ,
       -0.07238437,  0.03803752, -0.00064042,  0.05024209,  0.05024209]), 0.29714565802617193)


In [16]:
median_cleaned_x_0_lsq_SGD = least_squares_SGD(split_y[0], normalize(cleaned_with_median[0]), initial_w, 500, 0.001)

In [17]:
print(median_cleaned_x_0_lsq_SGD)

(array([-0.04836316, -0.10000091, -0.02673626,  0.01805247,  0.02964008,
        0.01805247, -0.01776028,  0.02912592,  0.03460342,  0.04191371,
        0.03274352,  0.03205406, -0.02365191,  0.03306001,  0.03103124,
       -0.00610077,  0.03342681, -0.05820995,  0.03231787,  0.03231787]), 0.3688807319654526)


In [95]:
median_cleaned_x_0_lsq = least_squares(split_y[0], normalize(cleaned_with_median[0]))

In [96]:
print(median_cleaned_x_0_lsq)

(array([ 1.19348797e-01, -3.20910781e-01, -7.11010616e-01, -3.61745390e+03,
        2.92434098e+01,  3.61754007e+03, -7.97192079e+00, -1.48451124e+01,
       -9.21399265e-01,  8.74154018e+00,  1.22199385e-02,  9.00441921e-03,
        8.51383385e+00,  1.42330751e-01,  1.82847018e-02, -8.31032487e-02,
       -1.72379531e-01,  3.24838123e-02, -9.72704906e+00, -9.72704906e+00]), 0.2599109628362648)


In [97]:
median_cleaned_x_0_ridge = ridge_regression(split_y[0], normalize(cleaned_with_median[0]), 0.037)

In [98]:
print(median_cleaned_x_0_ridge)

(array([ 0.06227524, -0.46079671, -0.13201699,  0.0411997 ,  0.05966671,
        0.04119971,  0.1212882 ,  0.04336166,  0.07570685,  0.32090765,
        0.05564915,  0.05486386, -0.05369323,  0.05732194,  0.05516295,
       -0.06514523,  0.05211816,  0.00402923,  0.05500482,  0.05500482]), 0.3046039439677196)


In [99]:
median_cleaned_x_0_log = logistic_regression(split_y[0], normalize(cleaned_with_median[0]), initial_w, 10000, 0.01)

/home/benoit/Documents/EPFL/Master/MA1/MachineLearning/project1/ML_course_project1/scripts/implementations.py:78: RuntimeWarning: overflow encountered in exp
  return 1.0/(1 + np.exp(-z))


KeyboardInterrupt: 

In [ ]:
print(median_cleaned_x_0_log)

## Features Expension

In [13]:
def build_poly(x, degree):
   
    x_extended = np.ones(shape=x.shape)

    for d in range (1, degree +1):
        x_extended = np.c_[x_extended, x**d]
        

    return x_extended

In [14]:
a = np.array([[1,2,3,4],[5,6,7,8]])
build_poly(a, 2)

array([[ 1.,  1.,  1.,  1.,  1.,  2.,  3.,  4.,  1.,  4.,  9., 16.],
       [ 1.,  1.,  1.,  1.,  5.,  6.,  7.,  8., 25., 36., 49., 64.]])

## Cross Validation

In [15]:
#method to split the training set into a (new) training set and a test set (same as in lab03)

def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8 
    you will have 80% of your data set dedicated to training 
    and the rest dedicated to testing
    """
    # set seed
    np.random.seed(seed)
 
    # split the data based on the given ratio

    training_nbr = int(x.shape[0] * ratio)
    indexes = np.random.choice(x.shape[0],training_nbr, replace=False)
    
    x_train = x[indexes]
    y_train = y[indexes]
    x_test = np.delete(x, indexes, axis = 0)
    y_test = np.delete(y, indexes, axis = 0)
    
    
    return x_train, y_train, x_test, y_test

In [21]:
#perform cross-validation 

def crossValidation(x, y, splitRatio, degrees, seed =1):
    
    x_train, y_train, x_test, y_test = split_data(x, y, splitRatio, seed)
    
    a_training = []
    a_testing = []
    weights = []
    degr = []
    
    # define parameter (just add more for loops if there are more parameters for the model)
    lambdas = np.arange(0,0.03,0.0005)
    
    for ind, lambda_ in enumerate(lambdas):
        
        for ind_d, d in enumerate(degrees):
            
            
            #perform polynomial feature expension
            x_test_poly = build_poly(x_test,d)
            x_train_poly = build_poly(x_train, d)
            
            #normalize data (DANGER: the test set must be normalized with the training set's mean and std)
            mean = np.mean(x_train_poly)
            std = np.std(x_train_poly)
            
            x_train_ready = (x_train_poly - mean) / std
            x_test_ready = (x_test_poly - mean) / std
           
            
            #Models
        
            #ideal : lambdas = np.arange(0,0.03,0.001)
            w_star, e_tr = ridge_regression(y_train,x_train_ready, lambda_)
        
            #ideal : lambdas = np.arange(0,0.3,0.1)
            #w_star, e_tr = logistic_regression(y_train, x_train_ready,np.ones(x_train_ready.shape[1])  ,400, lambda_)
        
            #don't usel least squares with lambda bigger than 0.35 ideal: lambdas = np.arange(0.001,0.13,0.01)
            #w_star, e_tr = least_squares_GD(y_train, x_train_ready,np.ones(x_train_ready.shape[1])  ,400, lambda_)    
            #w_star, e_tr = least_squares_SGD(y_train, x_train,np.ones(x_train.shape[1])  ,400, lambda_)
        
            #DON'T REALLY NEED TO DO CROSS VALIDATION FOR THIS ONE ;) BUT PRACTICAL TO RUN IT HERE
            #w_star, e_tr = least_squares(y_train, x_train_ready)  
        
            degr.append(d)
        
            #compare the prediction with the reality
            accuracy_training = np.count_nonzero(predict_labels(w_star, x_train_ready) + y_train)/len(y_train)
            accuracy_testing = np.count_nonzero(predict_labels(w_star, x_test_ready) + y_test)/len(y_test)
        
            a_training.append(accuracy_training)
            a_testing.append(accuracy_testing)
            weights.append(w_star)
            print("lambda={l:.5f},degree={deg}, Training Accuracy={tr}, Testing Accuracy={te}".format(
                   l=lambda_, tr=a_training[ind*len(degrees)+ind_d], te=a_testing[ind*len(degrees)+ind_d], deg=d))
        
            #plt.plot(lambdas, a_training,'r--' , lambdas, a_testing, 'g--')
            #plt.show
    
    return weights[np.argmax(a_testing)], degr[np.argmax(a_testing)], a_testing[np.argmax(a_testing)]

In [16]:
#perform cross-validation 

def crossValidationForLogistic_reg(x, y, splitRatio, degrees, seed =1):
    
    x_train, y_train, x_test, y_test = split_data(x, y, splitRatio, seed)
    
    a_training = []
    a_testing = []
    weights = []
    degr = []
    
    index = 0
    
    # define parameter (just add more for loops if there are more parameters for the model)
    lambdas = np.arange(0,0.3,0.01)
    gammas = np.arange(0,0.03,0.0005)
    
    for ind, lambda_ in enumerate(lambdas):
        
        for ind_d, d in enumerate(degrees):
            
            for ind_g, gamma in enumerate(gammas):
            
                #perform polynomial feature expension
                x_test_poly = build_poly(x_test,d)
                x_train_poly = build_poly(x_train, d)
            
                #normalize data (DANGER: the test set must be normalized with the training set's mean and std)
                mean = np.mean(x_train_poly)
                std = np.std(x_train_poly)
            
                x_train_ready = (x_train_poly - mean) / std
                x_test_ready = (x_test_poly - mean) / std
           
            
                #Model
        
                #ideal : lambdas = np.arange(0,0.03,0.001)
                w_star, e_tr = reg_logistic_regression(y_train, x_train_ready, lambda_, np.ones(x_test_ready.shape[1]), 400, gamma)
        
           
                degr.append(d)
        
                #compare the prediction with the reality
                accuracy_training = np.count_nonzero(predict_labels(w_star, x_train_ready) + y_train)/len(y_train)
                accuracy_testing = np.count_nonzero(predict_labels(w_star, x_test_ready) + y_test)/len(y_test)
        
                a_training.append(accuracy_training)
                a_testing.append(accuracy_testing)
                weights.append(w_star)
                print("lambda={l:.5f},degree={deg}, Training Accuracy={tr}, Testing Accuracy={te}".format(
                       l=lambda_, tr=a_training[index], te=a_testing[index], deg=d))
        
                #increment index
                index = index + 1
    
    return weights[np.argmax(a_testing)], degr[np.argmax(a_testing)], a_testing[np.argmax(a_testing)]

In [71]:
def put_together(labels, indices):
    
    #First build first chunk
    ids_0 = np.matrix(indices[0]).T
    lab_0 = np.matrix(labels[0]).T
    
    unsorted_res = np.concatenate((ids_0, lab_0), axis=1)
    
    for i in range(1,len(labels)):
        ids = np.matrix(indices[i]).T
        lab = np.matrix(labels[i]).T
        by_jet_num = np.concatenate((ids, lab), axis=1)
        unsorted_res = np.concatenate((unsorted_res, by_jet_num), axis=0)
    
    sorted_res = unsorted_res[np.lexsort(np.fliplr(unsorted_res).T)]
    
    return sorted_res[0,:,:][:,1]

## Generate predictions and save ouput in csv format for submission:

In [24]:
DATA_TEST_PATH = '../data/test.csv' 
y_donotUse, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [25]:
#separate data with respect to column 24 and remove None
split_x_test, _, split_ids_test =  separate(y_donotUse, tX_test, ids_test)


split_x_cleaned_test = removeNone(split_x_test, dataStatistics(split_x_test))

#median instead of None
split_x_with_median = putMedianInsteadOfNone(split_x_cleaned_test)

#line dropped when None
#split_x_drop_lines, split_y_dropped_split_indexes_dropped = dropLineIfNone(split_x_cleaned_test, _, split_ids_test)

#degrees for polynomial feature expension
degrees = [1,2,3,4,5,6,7,8]

y_res = []

acc = []

for i in range(4):
    #training
    w_star, d, accuracy = crossValidation(cleaned_with_median[i], split_y[i], 0.8, degrees ,6)
    
    #prediction
    y_res.append(predict_labels(w_star, normalize(build_poly(split_x_with_median[i], d))))


    acc.append(accuracy)

print("Accuracy per jet nbr: \n")
print(acc)


Statistics 
Type :
0
[0.2605448 0.        0.        0.        1.        1.        1.
 0.        0.        0.        0.        0.        1.        0.
 0.        0.        0.        0.        0.        0.        0.
 0.        0.        1.        1.        1.        1.        1.
 1.        0.       ]
Statistics 
Type :
1
[0.09834149 0.         0.         0.         1.         1.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         1.         0.        ]
Statistics 
Type :
2
[0.05881481 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.        ]
Statistics 
Type :
3
[0.06376737 0.         0.     

lambda=0.00250,degree=11, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00250,degree=12, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00300,degree=1, Training Accuracy=0.8020267734267484, Testing Accuracy=0.8085372566681679
lambda=0.00300,degree=2, Training Accuracy=0.7509821093456774, Testing Accuracy=0.7596456988440174
lambda=0.00300,degree=3, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00300,degree=4, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00300,degree=5, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00300,degree=6, Training Accuracy=0.7435756286750907, Testing Accuracy=0.7498373617574938
lambda=0.00300,degree=7, Training Accuracy=0.7435005629926186, Testing Accuracy=0.7496371916128709
lambda=0.00300,degree=8, Training Accuracy=0.7436006505692481, Testing Accuracy=0.7498373617574938
lambda=0

lambda=0.00600,degree=10, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00600,degree=11, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00600,degree=12, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00650,degree=1, Training Accuracy=0.8005254597773052, Testing Accuracy=0.807586448481209
lambda=0.00650,degree=2, Training Accuracy=0.7477417740522958, Testing Accuracy=0.7553920832707801
lambda=0.00650,degree=3, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00650,degree=4, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00650,degree=5, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00650,degree=6, Training Accuracy=0.7435756286750907, Testing Accuracy=0.7498373617574938
lambda=0.00650,degree=7, Training Accuracy=0.7435005629926186, Testing Accuracy=0.7496371916128709
lambda=0

lambda=0.00950,degree=9, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00950,degree=10, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00950,degree=11, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.00950,degree=12, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01000,degree=1, Training Accuracy=0.7996747153759539, Testing Accuracy=0.80628534254116
lambda=0.01000,degree=2, Training Accuracy=0.7464031027148755, Testing Accuracy=0.7524896161737477
lambda=0.01000,degree=3, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01000,degree=4, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01000,degree=5, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01000,degree=6, Training Accuracy=0.7435756286750907, Testing Accuracy=0.7498373617574938
lambda=0.

lambda=0.01300,degree=8, Training Accuracy=0.7436006505692481, Testing Accuracy=0.7498373617574938
lambda=0.01300,degree=9, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01300,degree=10, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01300,degree=11, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01300,degree=12, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01350,degree=1, Training Accuracy=0.7990491680220193, Testing Accuracy=0.8052844918180453
lambda=0.01350,degree=2, Training Accuracy=0.7449143000125109, Testing Accuracy=0.7507881699444527
lambda=0.01350,degree=3, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01350,degree=4, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=0.01350,degree=5, Training Accuracy=0.7436256724634055, Testing Accuracy=0.7498373617574938
lambda=

...

lambda=0.01400,degree=4, Training Accuracy=0.6993401387400597, Testing Accuracy=0.6999774419129258
lambda=0.01400,degree=5, Training Accuracy=0.6985505611640629, Testing Accuracy=0.700203022783668
lambda=0.01400,degree=6, Training Accuracy=0.698099373977779, Testing Accuracy=0.700203022783668
lambda=0.01400,degree=7, Training Accuracy=0.6979301787829225, Testing Accuracy=0.6999774419129258
lambda=0.01400,degree=8, Training Accuracy=0.6974225931983532, Testing Accuracy=0.6990751184299572
lambda=0.01400,degree=9, Training Accuracy=0.6970842028086403, Testing Accuracy=0.6986239566884728
lambda=0.01400,degree=10, Training Accuracy=0.6969714060120693, Testing Accuracy=0.6986239566884728
lambda=0.01400,degree=11, Training Accuracy=0.6966330156223564, Testing Accuracy=0.6990751184299572
lambda=0.01400,degree=12, Training Accuracy=0.6966330156223564, Testing Accuracy=0.698849537559215
lambda=0.01450,degree=1, Training Accuracy=0.7178388133777001, Testing Accuracy=0.7148657793819084
lambda=0.01

lambda=0.01750,degree=4, Training Accuracy=0.6994529355366308, Testing Accuracy=0.6999774419129258
lambda=0.01750,degree=5, Training Accuracy=0.6991709435452034, Testing Accuracy=0.7004286036544101
lambda=0.01750,degree=6, Training Accuracy=0.6987761547572049, Testing Accuracy=0.700203022783668
lambda=0.01750,degree=7, Training Accuracy=0.698099373977779, Testing Accuracy=0.6999774419129258
lambda=0.01750,degree=8, Training Accuracy=0.6977045851897806, Testing Accuracy=0.6995262801714415
lambda=0.01750,degree=9, Training Accuracy=0.6971969996052112, Testing Accuracy=0.6990751184299572
lambda=0.01750,degree=10, Training Accuracy=0.6969714060120693, Testing Accuracy=0.6986239566884728
lambda=0.01750,degree=11, Training Accuracy=0.6967458124189273, Testing Accuracy=0.698849537559215
lambda=0.01750,degree=12, Training Accuracy=0.6967458124189273, Testing Accuracy=0.6990751184299572
lambda=0.01800,degree=1, Training Accuracy=0.7181208053691275, Testing Accuracy=0.7144146176404241
lambda=0.0

lambda=0.02100,degree=4, Training Accuracy=0.6998477243246292, Testing Accuracy=0.6997518610421837
lambda=0.02100,degree=5, Training Accuracy=0.6996785291297727, Testing Accuracy=0.700203022783668
lambda=0.02100,degree=6, Training Accuracy=0.6993401387400597, Testing Accuracy=0.700203022783668
lambda=0.02100,degree=7, Training Accuracy=0.6984377643674919, Testing Accuracy=0.6999774419129258
lambda=0.02100,degree=8, Training Accuracy=0.6978173819863516, Testing Accuracy=0.6999774419129258
lambda=0.02100,degree=9, Training Accuracy=0.6973097964017821, Testing Accuracy=0.6990751184299572
lambda=0.02100,degree=10, Training Accuracy=0.6970278044103547, Testing Accuracy=0.6986239566884728
lambda=0.02100,degree=11, Training Accuracy=0.6968586092154982, Testing Accuracy=0.6983983758177307
lambda=0.02100,degree=12, Training Accuracy=0.6966894140206418, Testing Accuracy=0.6990751184299572
lambda=0.02150,degree=1, Training Accuracy=0.7177824149794145, Testing Accuracy=0.7150913602526505
lambda=0.

lambda=0.02450,degree=4, Training Accuracy=0.7002989115109131, Testing Accuracy=0.700203022783668
lambda=0.02450,degree=5, Training Accuracy=0.7001861147143421, Testing Accuracy=0.700203022783668
lambda=0.02450,degree=6, Training Accuracy=0.6998477243246292, Testing Accuracy=0.6999774419129258
lambda=0.02450,degree=7, Training Accuracy=0.6990017483503469, Testing Accuracy=0.6999774419129258
lambda=0.02450,degree=8, Training Accuracy=0.697986577181208, Testing Accuracy=0.6999774419129258
lambda=0.02450,degree=9, Training Accuracy=0.6975917883932097, Testing Accuracy=0.6993006993006993
lambda=0.02450,degree=10, Training Accuracy=0.6970842028086403, Testing Accuracy=0.698849537559215
lambda=0.02450,degree=11, Training Accuracy=0.6968022108172128, Testing Accuracy=0.6983983758177307
lambda=0.02450,degree=12, Training Accuracy=0.6968022108172128, Testing Accuracy=0.6990751184299572
lambda=0.02500,degree=1, Training Accuracy=0.7175568213862726, Testing Accuracy=0.715768102864877
lambda=0.025

lambda=0.02800,degree=4, Training Accuracy=0.7008064970954825, Testing Accuracy=0.7013309271373788
lambda=0.02800,degree=5, Training Accuracy=0.700750098697197, Testing Accuracy=0.7015565080081209
lambda=0.02800,degree=6, Training Accuracy=0.7001861147143421, Testing Accuracy=0.6997518610421837
lambda=0.02800,degree=7, Training Accuracy=0.6993401387400597, Testing Accuracy=0.700203022783668
lambda=0.02800,degree=8, Training Accuracy=0.6982685691726355, Testing Accuracy=0.6999774419129258
lambda=0.02800,degree=9, Training Accuracy=0.6977045851897806, Testing Accuracy=0.6995262801714415
lambda=0.02800,degree=10, Training Accuracy=0.6971969996052112, Testing Accuracy=0.6990751184299572
lambda=0.02800,degree=11, Training Accuracy=0.6968586092154982, Testing Accuracy=0.6983983758177307
lambda=0.02800,degree=12, Training Accuracy=0.6969150076137838, Testing Accuracy=0.698849537559215
lambda=0.02850,degree=1, Training Accuracy=0.717951610174271, Testing Accuracy=0.7162192646063614
lambda=0.02

In [73]:
OUTPUT_PATH = '../data/submission.csv'
#y_pred = predict_labels(weights, tX_test)
y_pred = put_together(y_res, split_ids_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)